In [ ]:
!pip install transformers datasets

In [ ]:
import torch
from transformers import GPT2LMHeadModel, PreTrainedTokenizerFast, Trainer, TrainingArguments
from datasets import load_dataset, Dataset
import pandas as pd
import numpy as np
from transformers import TrainerCallback

In [ ]:
from transformers import GPT2LMHeadModel, PreTrainedTokenizerFast

# kogpt 모델 로드
model_name = 'skt/kogpt2-base-v2' 
tokenizer = PreTrainedTokenizerFast.from_pretrained(model_name, bos_token='</s>', eos_token='</s>', unk_token='<unk>',
    pad_token='</s>')
model = GPT2LMHeadModel.from_pretrained(model_name)


In [ ]:
from google.colab import files

uploaded = files.upload()

# data.txt(텍스트 데이터셋) 파일 로드
with open("data.txt", "r", encoding="utf-8") as f:
    raw_data = f.read()

texts = [line.strip() for line in raw_data.split("\n") if line.strip()]


train_size = int(0.85 * len(texts))
train_texts = texts[:train_size]
test_texts = texts[train_size:]

train_dataset = Dataset.from_dict({'text': train_texts})
test_dataset = Dataset.from_dict({'text': test_texts})


In [ ]:
#토크나이징
def tokenize_function(examples):
    model_inputs = tokenizer(examples["text"], padding='max_length', max_length=512, truncation=True)
    model_inputs["labels"] = model_inputs["input_ids"].copy()
    return model_inputs

tokenized_train = train_dataset.map(tokenize_function, batched=True)
tokenized_test = test_dataset.map(tokenize_function, batched=True)

In [ ]:
# 파인튜닝1 
training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=10,
    per_device_train_batch_size=2,
    evaluation_strategy="no", 
    save_strategy="epoch",
    logging_strategy="epoch",
    logging_dir="./logs",
    logging_steps=10,
    save_total_limit=2,
    report_to="none",
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
)

trainer.train()

In [ ]:
import json
from google.colab import files
from datasets import Dataset

uploaded = files.upload()

# qadata.txt(질의응답 json형태 데이터셋) 파일 로드
with open("qadata.txt", "r", encoding="utf-8") as f:
    qa_data = json.load(f)

qa_dataset = Dataset.from_dict(qa_data)

train_testsplit = qa_dataset.train_test_split(test_size=0.15)
train_dataset = train_testsplit['train']
test_dataset = train_testsplit['test']


In [ ]:
#토크나이징
def preprocess_function(examples):
    inputs = [q + tokenizer.eos_token + a for q, a in zip(examples['question'], examples['answer'])]
    model_inputs = tokenizer(inputs, max_length=512, truncation=True, padding="max_length")
    model_inputs["labels"] = model_inputs["input_ids"].copy()
    return model_inputs

tokenized_train_dataset = train_dataset.map(preprocess_function, batched=True)
tokenized_test_dataset = test_dataset.map(preprocess_function, batched=True)


In [ ]:
#파인튜닝2
training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=5,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    evaluation_strategy="epoch", 
    save_strategy="epoch",
    logging_strategy="epoch",
    logging_dir="./logs",
    logging_steps=10,
    save_total_limit=2,
    load_best_model_at_end=True,
    report_to="none",
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_dataset,  
    eval_dataset=tokenized_test_dataset,   
)

trainer.train()


In [ ]:
# 테스트 예시
test_questions = [
        "의무유급제도란 무엇인가요?",
        "제적이란 무엇인가요?",
        "유급학기의 계절수업 성적은 어떻게 되나요?",
        "의무유급 학기의 성적은 어떻게 되나요?",
        "수료/수여대상자가 의무유급대상이 될 경우 어떻게 되나요?",
        "성적은 어떻게 평가되나요?",
        "매 학기에 정기시험은 몇회 있나요?",
        "A등급의 비율은 얼마로 제한되나요?",
        "상대평가 대상에서 제외되는 대상은 누구인가요?",
        "성적 확인이 제한되는 경우는 어떤 것이 있나요?",
        "학사경고 사항은 어디에 기재되나요?",
        "학사경고자는 어떤 조치가 따르나요?",
]

for question in test_questions:
    inputs = tokenizer.encode(question + tokenizer.eos_token, return_tensors='pt').to(model.device)
    outputs = model.generate(inputs, max_length=512, do_sample=True, top_k=50)
    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    answer = generated_text[len(question):].strip()
    print(f"질문: {question}")
    print(f"답변: {answer}")
    print("-" * 50)



In [ ]:
# loss 값을 저장할 클래스
class LossRecorder(TrainerCallback):
    def __init__(self):
        self.train_losses = []
        self.train_epochs = []
        self.eval_losses = []
        self.eval_epochs = []
    
    def on_log(self, args, state, control, logs=None, **kwargs):
        if logs is not None:
            if 'loss' in logs:
                self.train_losses.append(logs['loss'])
                self.train_epochs.append(state.epoch)
            if 'eval_loss' in logs:
                self.eval_losses.append(logs['eval_loss'])
                self.eval_epochs.append(state.epoch)

loss_recorder = LossRecorder()

In [ ]:
# 추가학습
training_args.num_train_epochs = 10

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_test_dataset,
    callbacks=[loss_recorder],
)

trainer.train()



In [ ]:
# 손실 값 가져오기
train_epochs = loss_recorder.train_epochs
train_losses = loss_recorder.train_losses
eval_epochs = loss_recorder.eval_epochs
eval_losses = loss_recorder.eval_losses

import matplotlib.pyplot as plt

plt.figure(figsize=(8, 6))
plt.plot(train_epochs, train_losses, label='Training Loss')
plt.plot(eval_epochs, eval_losses, label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training and Validation Loss Over Epochs')
plt.legend()
plt.grid(True)
plt.show()


In [ ]:
# 모델 평가 (BLEU)
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction

# BLEU 계산 함수
def calculate_bleu(references, hypotheses):
    smooth = SmoothingFunction().method1  # Smoothing 함수 설정
    bleu_scores = []
    for ref, hyp in zip(references, hypotheses):
        # BLEU-4 기준 점수 계산
        score = sentence_bleu([ref.split()], hyp.split(), smoothing_function=smooth)
        bleu_scores.append(score)
    return bleu_scores

# 모델 생성 답변 수집 함수
def generate_answers(model, tokenizer, questions):
    generated_answers = []
    for question in questions:
        inputs = tokenizer.encode(question + tokenizer.eos_token, return_tensors='pt').to(model.device)
        outputs = model.generate(inputs, max_length=512, do_sample=True, top_k=50)
        generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
        answer = generated_text[len(question):].strip()
        generated_answers.append(answer)
    return generated_answers

In [ ]:
# 테스트 질문
test_questions = [
    "의무유급제도란 무엇인가요?",
    "제적이란 무엇인가요?",
    "유급학기의 계절수업 성적은 어떻게 되나요?",
    "학기개시일 30일 이전에 일반휴학을 신청할 경우 얼마의 금액을 돌려받을 수 있나요?",
    "학기개시일 30일에서 60일이 경과한 경우 얼마의 금액을 돌려받을 수 있나요?",
    "창업휴학업무는 어디에서 담당하고 있나요?",
    "복학 신청은 어디에서 가능한가요?",
    "학석사 연계과정에는 어떤 것이 있나요?"
]

# 생성된 답변 얻기
generated_answers = generate_answers(model, tokenizer, test_questions)

# 참조 답변 (정답)
reference_answers = [
    "의무유급제도는 학업부진자에 대한 교육적 차원에서 학사경고 횟수에 따라 유급하도 록 하여 학생에게 학업 기회를 부여하는 제도입니다.",
    "제적은 자퇴, 학사경고(의무유급 2회 후 재차 연속3회, 통산4회 받은 경우), 휴학기 간 종료 후 미복학, 미등록 등으로 인하여 학적을 상실하는 것입니다.",
    "유급학기의 계절수업 및 해당학기 이후 휴학 중 계절수업을 통해 취득한 성적도 무효 처리됩니다.",
    "학기 개시일 30일 경과 전일 경우 입학금을 제외한 등록금 전액이 반환됩니다.",
    "학기 개시일 30일 경과한 날로부터 60일 경과 전일 경우 입학금을 제외한 등록금의 2/3에 해당하는 금액이 반환됩니다.",
    "창업휴학업무는 창업지원단에서 담당하고 있습니다.",
    "해당학기 학사일정에 정해진 휴·복학 기간 내 학사서비스(GLS)를 통해 복학 신청이 가능합니다.",
    "학사과정 졸업유형에 따라 조기졸업트랙과 정규졸업트랙 두 가지가 있습니다."
]

# 출력 확인
for i, (question, generated, reference) in enumerate(zip(test_questions, generated_answers, reference_answers)):
    print(f"질문 {i+1}: {question}")
    print(f"모델 출력 답변: {generated}")
    print(f"참조 답변: {reference}")
    print("-" * 50)

# BLEU 점수 계산 (1에 가까울수록 좋음)
bleu_scores = calculate_bleu(reference_answers, generated_answers)
print("BLEU 점수:")
for i, score in enumerate(bleu_scores):
    print(f"질문 {i+1}의 BLEU 점수: {score:.4f}")

# 베이스라인 모델 로드
baseline_model = GPT2LMHeadModel.from_pretrained(model_name)

for question in test_questions:
    inputs = tokenizer.encode(question + tokenizer.eos_token, return_tensors='pt')
    outputs = baseline_model.generate(inputs, max_length=50, do_sample=True, top_k=50)
    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    answer = generated_text[len(question):].strip()
    print(f"[베이스라인] 질문: {question}")
    print(f"[베이스라인] 답변: {answer[len(question):]}")
    print("-" * 50)
